## Image Drawing
This script uses an SVG file to draw an image on an 8.5x11" piece of paper. To get started, you should determine the global coordinates of your paper including x,y, and z. Assume that the end effector stays vertical the entire time. If you would like to generate your own SVG files, checkout the [linedraw](https://github.com/LingDong-/linedraw) repository.

In [ ]:
# Initiate ROS2 and MyUR3e Class
from myur import MyUR3e

robot = MyUR3e()

In [ ]:
# First close the gripper on the pen
robot.move_gripper(100)

# The use freedrive to measure the four corners of the page.

# Corners of the page in global frame
LT = [-0.0904, 0.4208, 0.22]  # Left Top
LB = [-0.0904, 0.2049, 0.217]  # Left Bottom
RB = [0.1890, 0.2049, 0.217]  # Right Bottom
RT = [0.1890, 0.4208, 0.22]  # Right Top

PAPER_WIDTH = 0.2159  # meters, or 8.5"
PAPER_HEIGHT = 0.2794  # meters, or 11"

In [ ]:
# Function to convert paper coordinates to global robot coordinates
def paper_coordinates(x, y, up=True):
    cords = []
    bounds = [x > 0.PAPER_HEIGHT,y > 0.PAPER_WIDTH,x < 0,y < 0]
    if True in bounds:
        raise RuntimeError("Your coordinates are off the paper")
    cords.append(LB[0] + x) # zero x at bottom left
    cords.append(LB[1] + y) # zero y at bottom left
    if up:
        z = max(LT[2],LB[2],RB[2],RT[2])
    else: # add Z height, assume paper is not perfectly aligned with robot's axes.
        # Calculate the interpolation factors
        t = (x - LB[0]) / (RB[0] - LB[0])  # Horizontal interpolation factor
        u = (y - LB[1]) / (LT[1] - LB[1])  # Vertical interpolation factor

        # Interpolate along the x-axis
        z_bottom = (1 - t) * LB[2] + t * RB[2]
        z_top = (1 - t) * LT[2] + t * RT[2]

        # Interpolate along the y-axis
        z = (1 - u) * z_bottom + u * z_top

    return cords + [z, 0, 0, 0]


# Example use: pen at (0.1,0.1) on the page with the pen up
print(paper_coordinates(0.1, 0.1, up=True))

In [ ]:
import xml.etree.ElementTree as ET

# If you are using a different SVG, update it here
svg_file = "img/crogers.svg" # (make sure it was made with the linedraw program, other SVGs are formatted differently)


# Create a list of lines containing coordinates from SVG file
def svg_polyline_to_coordinates(svg_file):
    tree = ET.parse(svg_file)
    root = tree.getroot()

    # Namespace handling
    namespaces = {"svg": "http://www.w3.org/2000/svg"}

    polylines = root.findall(".//svg:polyline", namespaces)
    all_coordinates = []

    max_x = float("-inf")
    max_y = float("-inf")

    for polyline in polylines:
        points = polyline.get("points")
        if points:
            # Split the points string into individual coordinate pairs
            points_list = points.strip().split(",")
            # Extract x and y coordinates by pairing adjacent items in the list
            coordinates = [
                (float(points_list[i]), float(points_list[i + 1]))
                for i in range(0, len(points_list), 2)
            ]
            all_coordinates.append(coordinates)

            # Update max x and y values
            for x, y in coordinates:
                if x > max_x:
                    max_x = x
                if y > max_y:
                    max_y = y

    return all_coordinates, (max_x, max_y)


lines, max_vals = svg_polyline_to_coordinates(svg_file)

In [ ]:
# height of paper is 0.2159m
# width of paper is 0.2794m
draw_height = 0.18  # meters
draw_width = 0.25  # meters
scale_x = draw_width / max_vals[0]
scale_y = draw_height / max_vals[1]
scale = min(scale_x, scale_y)

vis_trajectory = []  # collection of all trajectories for plot
for line in lines:
    drawing_trajectory = []
    for i, position in enumerate(line):
        y = 0.2159 - (position[1] * scale + ((0.2159 - draw_height) / 2))
        x = position[0] * scale + ((0.2794 - max_vals[0] * scale) / 2)

        if i == 0:
            drawing_trajectory.append(
                paper_coordinates(x, y, up=True)
            )  # pen up at first point
            drawing_trajectory.append(
                paper_coordinates(x, y, up=False)
            )  # pen down at first point
        else:
            drawing_trajectory.append(
                paper_coordinates(x, y, up=False)
            )  # interpolated points
    drawing_trajectory.append(paper_coordinates(x, y, up=True))  # pen up at last point
    vis_trajectory.extend(drawing_trajectory)

    # This line actually moves the robot
    robot.move_global(drawing_trajectory, time=("cv", 0.1, 0.075))

In [ ]:
from IPython.display import HTML
robot.clear_vis()
robot.move_global(vis_trajectory, vis_only=True)
HTML(filename="ur3e_trajectory.html")